In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# CONFIG
MODEL_PATH = "/Users/drago/plant_detector/plant_model.h5"
DATASET_ROOT = "/Users/drago/plant_detector/leafsnap-dataset"
METADATA_PATH = f"{DATASET_ROOT}/leafsnap-dataset-images.txt"
IMG_SIZE = 224

# Load model
model = load_model(MODEL_PATH)
print("✅ Model loaded!")

# Load metadata to get sample test images
df = pd.read_csv(METADATA_PATH, sep='\t')

# Clean/validate image paths
valid_images = []
for _, row in df.iterrows():
    img_path = os.path.join(DATASET_ROOT, row['image_path'])
    seg_path = os.path.join(DATASET_ROOT, row['segmented_path'])
    if os.path.exists(img_path):
        valid_images.append((img_path, row['species']))
    elif os.path.exists(seg_path):
        valid_images.append((seg_path, row['species']))

# Take first N test cases
test_samples = valid_images[:5]

# Label encoder
species_list = sorted(list(set([s for _, s in valid_images])))
species_to_idx = {s: i for i, s in enumerate(species_list)}
idx_to_species = {i: s for s, i in species_to_idx.items()}

# Prediction function
def predict_image(path):
    raw = tf.io.read_file(path)
    is_jpeg = tf.strings.regex_full_match(path, ".*\.jpe?g")
    image = tf.cond(
        is_jpeg,
        lambda: tf.image.decode_jpeg(raw, channels=3),
        lambda: tf.image.decode_png(raw, channels=3)
    )
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = tf.expand_dims(image, 0)
    prediction = model.predict(image)
    predicted_index = tf.argmax(prediction, axis=1).numpy()[0]
    predicted_species = idx_to_species[predicted_index]
    return predicted_species

# Run predictions
print("\n🔍 Running predictions on sample images...\n")
for path, true_species in test_samples:
    predicted = predict_image(path)
    print(f"Image: {os.path.basename(path)}")
    print(f"🔹 Actual:    {true_species}")
    print(f"🔸 Predicted: {predicted}")
    print("-" * 40)


✅ Model loaded!

🔍 Running predictions on sample images...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
Image: ny1157-01-1.jpg
🔹 Actual:    Abies concolor
🔸 Predicted: Picea pungens
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: ny1157-01-2.jpg
🔹 Actual:    Abies concolor
🔸 Predicted: Picea pungens
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: ny1157-01-3.jpg
🔹 Actual:    Abies concolor
🔸 Predicted: Abies concolor
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: ny1157-01-4.jpg
🔹 Actual:    Abies concolor
🔸 Predicted: Abies concolor
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: ny1157-02-1.jpg
🔹 Actual:    Abies concolor
🔸 Predicted: Abies concolor
----------------------------------------
